# MorphoXAI – Stage 1: Morphologic Spectrum Construction – Starter Notebook


In [ ]:
from pathlib import Path
import pandas as pd
import sys

# project root = one level up from this notebook
PROJECT_ROOT = Path("..").resolve()
print("Project root:", PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Directory containing the metadata for all WSIs used to train the 
# full-data MIL model that will be explained in later steps.
DATA_SPLIT_DIR = PROJECT_ROOT / "Data_Split" 

# Directory used to store all WSI feature bags (extracted by CONCH), one H5 file per slide.
FEATURE_BAG_DIR = PROJECT_ROOT / "feature_bags"

DATA_SPLIT_DIR.mkdir(exist_ok=True)
FEATURE_BAG_DIR.mkdir(exist_ok=True)

# Configuration file specifying label mappings, model settings, 
# and hyperparameters used throughout the MorphoXAI pipeline.
CONFIG_PATH = PROJECT_ROOT / "Morphologic_Spectrum_Construction" / "config.yaml"

DATA_SPLIT_DIR, FEATURE_BAG_DIR, CONFIG_PATH

## Step 1 – Data preparation

In [ ]:
metadata_csv = DATA_SPLIT_DIR / "output_svs_file_mapping.csv" #the metadat csv file for all WSIs
assert metadata_csv.exists(), f"Metadata CSV not found: {metadata_csv}"

df_meta = pd.read_csv(metadata_csv)
df_meta.head()

In [ ]:
from Morphologic_Spectrum_Construction import run_patient_level_split

mode = "random"
split_num = 5 #number of patient permutations
fold_num = 10 #number of folds
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
label_config = CONFIG_PATH

output_paths = run_patient_level_split(
    data_csv=metadata_csv,
    output_dir=DATA_SPLIT_DIR,
    label_config=label_config,
    mode=mode,
    split_num=split_num,
    fold_num=fold_num,
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    test_ratio=test_ratio,
)

output_paths

The resulting data splits are stored in DATA_SPLIT_DIR, named in the format `Datasplit_{s_index}_{n_folds}_fold_by_patient.csv`

## Step 2 – WSI → feature bags (CONCH embeddings)

In [ ]:
# Select an example WSI from the metadata (here we use the first row; feel free to choose another one)
row = df_meta.iloc[0]
row
# WSI_ROOT is the top-level directory where the original whole-slide images (WSIs) are stored
WSI_ROOT = Path("/path/to/wholeslides").resolve()
print(WSI_ROOT)
wsi_path = WSI_ROOT / "xxx.svs"
wsi_path = wsi_path.resolve()
print(wsi_path, wsi_path.exists())

In [ ]:
tile_size = 360
out_size = 224
batch_size = 256
num_workers = 8

from Morphologic_Spectrum_Construction import preprocess_wsi_to_h5

result = preprocess_wsi_to_h5(
    input_slide=wsi_path,
    output_dir=FEATURE_BAG_DIR,
    tile_size_microns=360,
    out_size=224,
    batch_size=256,
    workers=8,
    hf_auth_token="", #put your hugging face token here
)

result

For each processed WSI, two types of files are produced in the `FEATURE_BAG_DIR`: `{Slide_ID}.h5`, `{Slide_ID}_features_QC.png`.

## Step 3 – MIL training with CLAM 


In [ ]:
s_index = 0          # which split index (patient permutation) from Data Preparation
n_folds = 10         # must match the value used in Data Preparation

manifest_name = f"Datasplit_{s_index}_{n_folds}_fold_by_patient.csv"
manifest_path = DATA_SPLIT_DIR / manifest_name

print("Manifest path:", manifest_path)
print("Exists:", manifest_path.exists())

feature_bag_dir = Path("/path/to/feature_bags")
print("Feature bag dir:", feature_bag_dir, "Exists:", feature_bag_dir.exists())

#Directory where training checkpoints will be saved
out_checkpoint_dir = PROJECT_ROOT/ "runs"

In [ ]:
from Morphologic_Spectrum_Construction import train_single_round
round_idx = 0

train_single_round(
    manifest=manifest_path,
    feature_bag_dir=feature_bag_dir,
    out_checkpoint_dir=out_checkpoint_dir,
    round_idx=0,                   
    workers=4,
    full_training_index=None,      
    config_path=CONFIG_PATH,
)

Checkpoints are saved under `./runs/` in subfolders named according to the data split and the round index, e.g.:

  ```
  runs/Datasplit_0_10_fold_by_patient_random/round_0_small_random_hp1/
  ```
This example corresponds to the best model obtained from the first shuffle (`Datasplit_0_10_fold_by_patient_random`) during round 0 of cross-validation.

Each subfolder contains the **best model checkpoint** (selected based on validation performance) in PyTorch format:
  ```
  {round}_{epoch}_checkpoint.pt
  ```

## Step 4 – MIL testing (evaluation on held-out test sets)

In [ ]:
# manifest from Step 1 (same as training)
s_index = 0
n_folds = 10
manifest_name = f"Datasplit_{s_index}_{n_folds}_fold_by_patient.csv"
manifest_path = DATA_SPLIT_DIR / manifest_name
print("Manifest:", manifest_path, "Exists:", manifest_path.exists())

# feature bags (same directory used for training)
feature_bag_dir = Path("/path/to/feature_bags")
print("Feature bag dir:", feature_bag_dir, "Exists:", feature_bag_dir.exists())

# checkpoints directory = training outputs (./runs)
checkpoint_dir = PROJECT_ROOT/ "runs"
print("Checkpoint dir:", checkpoint_dir, "Exists:", checkpoint_dir.exists())


In [ ]:
from Morphologic_Spectrum_Construction import evaluate_all_rounds

evaluate_all_rounds(
    manifest=manifest_path,
    feature_bag_dir=feature_bag_dir,
    checkpoint_dir=checkpoint_dir,
    round_num=1, # number of CV rounds to evaluate
    workers=4,
    config_path=CONFIG_PATH,
)

All outputs are organized under `./test_result/{dataset_name}/` for each dataset, where `{dataset_name}` corresponds to the CSV manifest used (e.g. `Datasplit_0_10_fold_by_patient`). **ROC curve figures** are stored under `./ROC_Curve/{dataset_name}/`.

## Step 5 – Sample Stratification

In [ ]:
from Morphologic_Spectrum_Construction import run_sample_stratification

run_sample_stratification(
    test_result_root=PROJECT_ROOT/ "test_result",
    metadata_csv=DATA_SPLIT_DIR / "output_svs_file_mapping.csv",
    out_dir=PROJECT_ROOT/ "test_result",
    config_path=CONFIG_PATH
)

The output `sample_stratification_result.csv` is saved under `out_dir`.

## Step 6 – High-contribution Patches Exatraction

In [ ]:
# Directory containing all CONCH feature bags (*.h5)
feature_bag_dir = Path("/path/to/feature_bags")
print("Feature bag dir:", feature_bag_dir, "Exists:", feature_bag_dir.exists())

#Sample stratification CSV (output of Step 5)
sample_stratf_file=PROJECT_ROOT / "test_result" / "sample_stratification_result.csv"

#Directory where final high-contribution patches will be saved
output_dir=PROJECT_ROOT / "test_result" / "high_contribution_patches"

#Directory where MIL model checkpoints are stored (output of Step 3)
runs_root=PROJECT_ROOT / "runs"

#Directory containing all Datasplit_*.csv files (output of Step 1)
datasplit_root = PROJECT_ROOT / "Data_Split"
datasplit_root


In [ ]:
from Morphologic_Spectrum_Construction import extract_high_contribution_patches
coords_csv = extract_high_contribution_patches(
    feature_bag_dir=feature_bag_dir,
    sample_stratf_file=sample_stratf_file,
    output_dir=output_dir,
    subtype="Clear",  # Short class name; must match LABEL_MAP_SHORT in config.yaml
    stability="Consistently_Correct",
    topk_threshold=0.9,
    config_path=CONFIG_PATH,
    runs_root=runs_root,
    datasplit_root=datasplit_root,
    n_splits=5,
    n_folds=10,
)
coords_csv

The final results are written to a single CSV file named `{subtype}_{stability}.csv` in the directory specified by `output_dir`.

## Step 7 – High-contribution Patches Clustering

The script supports two modes:
1. **`evaluate_clusters`**  
   Run consensus clustering on a micro-clustered subset of embeddings to evaluate
   a list of candidate `k` values. This helps us decide a reasonable number of clusters.
2. **`export`**  
   Given a chosen `k` (number of clusters), perform hierarchical clustering on
   all patches, assign each patch an `hc_label`, and export.

### Mode 1 — `evaluate_clusters`

In [ ]:
# 1) Metadata CSV (slide_id → patient / tissue mapping)
metadata_csv = PROJECT_ROOT / "Data_Split" / "output_svs_file_mapping.csv"
print("metadata_csv:", metadata_csv, "| Exists:", metadata_csv.exists())

# 2) High-contribution patch CSV to cluster
#    This is the output from high_contribution_patches_extraction (Step 6),
#    typically named {subtype}_{stability}_all.csv
high_contri_csv = PROJECT_ROOT / "test_result" / "high_contribution_patches" / "Clear_Consistently_Correct_all.csv"
print("high_contri_csv:", high_contri_csv, "| Exists:", high_contri_csv.exists())

# 3) Disease / subtype label (short name, must match the 'label' column in high_contri_csv)
disease = "Clear"   # e.g., "Clear", "Endo", ...

# 4) Output directory for clustering results
cluster_out_dir = PROJECT_ROOT / "high_contribution_cluster"
cluster_out_dir.mkdir(parents=True, exist_ok=True)
print("cluster_out_dir:", cluster_out_dir, "| Exists:", cluster_out_dir.exists())

# 5) Feature bag directory (only required in 'export' mode)
feature_bag_dir = Path("/path/to/feature_bags")
print("Feature bag dir:", feature_bag_dir, "Exists:", feature_bag_dir.exists())


In [ ]:
from Morphologic_Spectrum_Construction import cluster_high_contribution_patches

# Candidate k values to evaluate
cluster_list = [3, 4, 5, 6, 7, 8]

# Consensus clustering hyperparameters
reps = 30      # number of subsampling runs
p_item = 0.8   # subsampling fraction per run
n_micro = 1000 # number of FAISS micro-centroids

cluster_high_contribution_patches(
    metadata_csv=metadata_csv,
    high_contri_csv=high_contri_csv,
    disease=disease,
    mode="evaluate_clusters",
    out_dir=cluster_out_dir,
    feature_bag_dir=None,     # Not used in this mode
    cluster_list=cluster_list,
    reps=reps,
    p_item=p_item,
    n_micro=n_micro,
    n_clusters=None,          # Ignored in this mode
)

Two Outputs would be saved in `out_dir`:
- `consensus_k_selection.csv` — table of `k` vs. cophenetic correlation (higher is better).
- `consensus_heatmap_k{k}.png` — consensus matrices visualizing clustering stability.

### Mode 2 — `export`

In [ ]:
# Suppose we decided that k = 5 is a good choice from evaluate_clusters
chosen_k = 5

from Morphologic_Spectrum_Construction import cluster_high_contribution_patches

cluster_high_contribution_patches(
    metadata_csv=metadata_csv,
    high_contri_csv=high_contri_csv,
    disease=disease,
    mode="export",
    out_dir=cluster_out_dir,
    feature_bag_dir=feature_bag_dir,  # Required in export mode
    cluster_list=None,                # Ignored here
    reps=reps,
    p_item=p_item,
    n_micro=n_micro,
    n_clusters=chosen_k,              # <-- key parameter
)

The results are stored in `out_dir\slide_results`. For each slide, a file named `<slide_id>_hc_coords.csv` is generated, containing patch-level coordinates assigned to each hierarchical cluster (`hc_label`). These files can be directly imported into the **MorphoCA** QuPath plugin for visualization. For detailed instructions, please refer to `./MorphoSpectrum-MIL/plugins/MorphoCA Manual.pdf`. 

## Step 8 – Export Morphologic Spectrum Stats
This step consolidates all information needed to build the morphologic spectrum, and saves it as a single .pkl file. The exported spectrum stats will be used directly in MorphoXAI Stage 2 for spectrum-based explanations on the independent test set.

In [ ]:
from Morphologic_Spectrum_Construction.export_spectrum import export_spectrum_stats
import yaml

with open(CONFIG_PATH) as f:
    CFG = yaml.safe_load(f)

algo_cfg = CFG["algorithm"]

# ---------------------------------------------------------------------
# Step 8 inputs are DIRECTLY linked to outputs from Step 6 and Step 7.
#
# Conventions:
# - Step 6 (high-contribution patch extraction) outputs:
#     {hc_patch_dir}/{group}_{stability}_all.csv
# - Step 7 (clustering export) outputs:
#     {cluster_out_dir}/{group}/slide_results/
# ---------------------------------------------------------------------

# Roots for Step 6 and Step 7 outputs
hc_patch_dir = PROJECT_ROOT / "test_result" / "high_contribution_patches"    # Step 6 outputs
cluster_out_dir = PROJECT_ROOT / "high_contribution_cluster"                 # Step 7 outputs

# -----------------------------
# Core patterns (Consistently_Correct slides)
# -----------------------------
# Step 6 outputs: patch tables with embeddings
core_embeds = {
    "Clear":  hc_patch_dir / "Clear_Consistently_Correct_all.csv",
    "Endo":   hc_patch_dir / "Endo_Consistently_Correct_all.csv",
    "High":   hc_patch_dir / "High_Consistently_Correct_all.csv",
    "Border": hc_patch_dir / "Border_Consistently_Correct_all.csv",
}

# Step 7 outputs: QuPath-ready patch coordinate exports (per slide)
core_clusters = {
    "Clear":  cluster_out_dir / "Clear"  / "slide_results",
    "Endo":   cluster_out_dir / "Endo"   / "slide_results",
    "High":   cluster_out_dir / "High"   / "slide_results",
    "Border": cluster_out_dir / "Border" / "slide_results",
}

# -----------------------------
# Transitional patterns (Highly_Variable slides)
# -----------------------------
# Step 6 outputs: patch tables with embeddings
transition_embeds = {
    "Endo_to_High": hc_patch_dir / "Endo_to_High_Highly_Variable_all.csv",
    "High_to_Endo": hc_patch_dir / "High_to_Endo_Highly_Variable_all.csv",
}

# Step 7 outputs: QuPath-ready patch coordinate exports (per slide)
transition_clusters = {
    "Endo_to_High": cluster_out_dir / "Endo_to_High" / "slide_results",
    "High_to_Endo": cluster_out_dir / "High_to_Endo" / "slide_results",
}

# -----------------------------
# Step 8 output
# -----------------------------
SPECTRUM_STATS_PATH = PROJECT_ROOT / "morphologic_spectrum_stats.pkl"

export_spectrum_stats(
    core_embeds=core_embeds,
    core_clusters=core_clusters,
    transition_embeds=transition_embeds,
    transition_clusters=transition_clusters,
    algorithm_cfg=algo_cfg,
    output_path=SPECTRUM_STATS_PATH,
)
